In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
from src.power_api import PowerAPI

In [2]:
df = pd.read_csv('data/table_mountain_sightings.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   76 non-null     int64  
 1   uuid                 76 non-null     object 
 2   observed_on          76 non-null     object 
 3   time_observed_at     69 non-null     object 
 4   user_id              76 non-null     int64  
 5   created_at           76 non-null     object 
 6   quality_grade        76 non-null     object 
 7   image_url            76 non-null     object 
 8   place_guess          76 non-null     object 
 9   latitude             76 non-null     float64
 10  longitude            76 non-null     float64
 11  positional_accuracy  64 non-null     float64
 12  private_place_guess  0 non-null      float64
 13  scientific_name      76 non-null     object 
 14  common_name          76 non-null     object 
dtypes: float64(4), int64(2), object(9)
memory 

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,76.0,1.183254e+08,8.397588e+07,1.018298e+07,4.316150e+07,1.137228e+08,1.720987e+08,2.954555e+08
user_id,76.0,2.775527e+06,2.508775e+06,1.391600e+04,7.724588e+05,1.670085e+06,4.318887e+06,9.217156e+06
latitude,76.0,-3.399927e+01,4.705669e-02,-3.415517e+01,-3.401066e+01,-3.399283e+01,-3.397006e+01,-3.386856e+01
longitude,76.0,1.843274e+01,2.935766e-02,1.838860e+01,1.841307e+01,1.843135e+01,1.844508e+01,1.854754e+01
positional_accuracy,64.0,1.147500e+02,3.079892e+02,1.000000e+00,4.000000e+00,9.000000e+00,5.875000e+01,1.798000e+03
private_place_guess,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.head()

,id,uuid,observed_on,time_observed_at,user_id,created_at,quality_grade,image_url,place_guess,latitude,longitude,positional_accuracy,private_place_guess,scientific_name,common_name
0,10182975,6387bfaa-b5ee-4289-a58c-b24e92ae9c5d,2018-03-11,2018-03-11 08:29:28 UTC,768969,2018-03-11 13:34:56 UTC,research,https://inaturalist-open-data.s3.amazonaws.com...,"Table Mountain, ZA-WC-CT, ZA-WC, ZA",-33.995726,18.413065,10.0,NaN,Pyracantha angustifolia,Narrowleaf Firethorn
1,10868145,b474337b-0c35-4095-b1cb-601982f866f4,2013-04-18,NaN,383144,2013-05-16 00:18:38 UTC,research,https://inaturalist-open-data.s3.amazonaws.com...,Alphen Trail Constantia,-34.008202,18.439436,1.0,NaN,Pyracantha angustifolia,Narrowleaf Firethorn
2,10889524,44dac2bc-a751-43dd-aa60-1cc06b8d13cc,2013-10-26,NaN,383144,2013-10-30 05:58:01 UTC,research,https://inaturalist-open-data.s3.amazonaws.com...,Constantiaberg Green Belt Alphen Trail - upper...,-34.006904,18.438363,1.0,NaN,Pyracantha angustifolia,Narrowleaf Firethorn
3,10921421,ef8f234d-6aa4-438e-a78e-23eb1de969ad,2013-06-13,NaN,773622,2013-12-29 13:21:33 UTC,research,https://inaturalist-open-data.s3.amazonaws.com...,"M64, Table Mountain National Park",-34.087017,18.424374,1.0,NaN,Pyracantha angustifolia,Narrowleaf Firethorn
4,11137123,a453d5c6-2166-4dd4-a610-a43a39802d33,2015-05-24,NaN,383144,2015-06-05 21:24:42 UTC,research,https://inaturalist-open-data.s3.amazonaws.com...,Path to Vlakkenberg above Constantia Neck: Vla...,-34.013778,18.404901,1.0,NaN,Pyracantha angustifolia,Narrowleaf Firethorn


In [6]:
coordinates_df = df[['id', 'latitude', 'longitude']].dropna().copy()
coordinates_df['id'] = coordinates_df['id'].astype(str)

# Output folder for individual weather files
OUTPUT_DIR = Path("data/weather_10yr")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Date range: last 10 years from today
end = pd.Timestamp.today().normalize()
start = end - pd.DateOffset(years=10)

start_date_str = start.strftime('%Y%m%d')
end_date_str = end.strftime('%Y%m%d')

# Fetch weather data for each coordinate
for idx, row in coordinates_df.iterrows():
    obs_id = row['id']
    lat = row['latitude']
    lon = row['longitude']

    output_file = OUTPUT_DIR / f"{obs_id}_{start_date_str}_{end_date_str}.csv"
    if output_file.exists():
        print(f"[{obs_id}] Already exists. Skipping.")
        continue

    try:
        print(f"[{obs_id}] Fetching weather from {start_date_str} to {end_date_str} at ({lat}, {lon})")

        weather = PowerAPI(
            start=start,
            end=end,
            long=lon,
            lat=lat
        )
        weather_df = weather.get_weather()

        if not weather_df.empty:
            weather_df.to_csv(output_file, sep=";")
            print(f"[{obs_id}] Saved to {output_file}")
        else:
            print(f"[{obs_id}] No weather data returned.")

    except Exception as e:
        print(f"[{obs_id}] Error: {e}")

# Now combine all files into one DataFrame
INPUT_DIR = OUTPUT_DIR  # Directory where all files are saved
OUTPUT_FILE = Path("data/all_weather_10yr_combined.csv")

all_weather_dfs = []

for file in INPUT_DIR.glob("*.csv"):
    try:
        weather_df = pd.read_csv(file, sep=';')
        obs_id = file.stem.split('_')[0]
        weather_df['id'] = obs_id

        # Merge coordinates to each weather data row
        merged_df = weather_df.merge(coordinates_df, on='id', how='left')
        all_weather_dfs.append(merged_df)

    except Exception as e:
        print(f"Error reading {file.name}: {e}")

if all_weather_dfs:
    final_df = pd.concat(all_weather_dfs, ignore_index=True)
    final_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Combined weather data saved to {OUTPUT_FILE}")
else:
    print("No weather data found to combine.")

[10182975] Fetching weather from 20150724 to 20250724 at (-33.9957259689, 18.413065318)
[10182975] Saved to data\weather_10yr\10182975_20150724_20250724.csv
[10868145] Fetching weather from 20150724 to 20250724 at (-34.00820232, 18.43943596)
[10868145] Saved to data\weather_10yr\10868145_20150724_20250724.csv
[10889524] Fetching weather from 20150724 to 20250724 at (-34.00690382, 18.43836308)
[10889524] Saved to data\weather_10yr\10889524_20150724_20250724.csv
[10921421] Fetching weather from 20150724 to 20250724 at (-34.0870168, 18.4243743)
[10921421] Saved to data\weather_10yr\10921421_20150724_20250724.csv
[11137123] Fetching weather from 20150724 to 20250724 at (-34.01377823, 18.40490089)
[11137123] Saved to data\weather_10yr\11137123_20150724_20250724.csv
[11168532] Fetching weather from 20150724 to 20250724 at (-33.9918549, 18.4140894)
[11168532] Saved to data\weather_10yr\11168532_20150724_20250724.csv
[12331342] Fetching weather from 20150724 to 20250724 at (-34.0092162, 18.416